In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
import numpy as np

#### Search your course and country in the studyportal.com, then copy the results link and paste here
It may take much longer according to your internet condition and server trafic so wait until the page is loaded completely.

In [3]:
url = input('Url: ') 
driver = webdriver.Safari()
driver.get(url)
time.sleep(3)

Url: https://www.mastersportal.com/search/master/data-science-big-data/united-states


In [4]:
field = url.split('/')[-2]
country = url.split('/')[-1]
country

'united-states'

In [5]:
no_change_counter = 0
old_height = 0
while True:
    # Try
    new_height = driver.execute_script("return document.body.scrollHeight") 
    driver.execute_script("window.scrollTo(0,{})".format(new_height))
    time.sleep(1)
    if new_height > old_height:
        old_height = new_height
    else:
        time.sleep(3)
        no_change_counter += 1
        
    if no_change_counter >= 3:
        break

In [6]:
results = driver.find_elements(By.XPATH, value = '//a[@class="SearchStudyCard js-bestFitStudycard"]')
len(results)

526

In [7]:
names_list = []
fees_list = []
fees_desc_list = []
duration = []
organization = []
location = []
summary = []
for i in range(len(results)):
    names_list.append(results[i].find_element(By.CLASS_NAME, value='StudyName').text)
    try:
        fees_list.append(int(results[i].find_element(By.CLASS_NAME, value='TuitionValue').text.split(' ')[0].replace(',', '')))
        fees_desc_list.append(results[i].find_element(By.CLASS_NAME, value='TuitionValue').text.split(' ', 1)[1])
        duration.append(results[i].find_element(By.CLASS_NAME, value='DurationValue').text)
        organization.append(results[i].find_element(By.CLASS_NAME, value='OrganisationName').text)
        location.append(results[i].find_element(By.CLASS_NAME, value='OrganisationLocation').text)
        summary.append(results[i].find_element(By.CLASS_NAME, value='Summary').text)
    except:
        fees_list.append(np.nan)
        fees_desc_list.append(np.nan)
        duration.append(np.nan)
        organization.append(np.nan)
        location.append(np.nan)
        summary.append(np.nan)


In [8]:
df = pd.DataFrame(list(zip(names_list, fees_list, fees_desc_list, duration, organization,location, summary)),
                 columns=['Courses', 'Fees', 'Fees Type', 'Duration', 'Organization', 'Location', 'Course Description'], index=None)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Courses             526 non-null    object 
 1   Fees                506 non-null    float64
 2   Fees Type           506 non-null    object 
 3   Duration            506 non-null    object 
 4   Organization        506 non-null    object 
 5   Location            506 non-null    object 
 6   Course Description  506 non-null    object 
dtypes: float64(1), object(6)
memory usage: 28.9+ KB


In [10]:
df.head()

,Courses,Fees,Fees Type,Duration,Organization,Location,Course Description
0,Data Science for Business Leaders,758.0,EUR / full,2 months,School of Business,Online,Master the strategic decision-making skills fo...
1,Applied Data Science,31222.0,EUR / year,1 year,Coursera - University of Michigan,Online,The Master of Applied Data Science degree offe...
2,Master's in Business Analytics,52176.0,EUR / year,1 year,Hult International Business School,Multiple locations,Harness the power of big data through statisti...
3,Computer Science,9487.0,EUR / year,1½ years,Coursera - Arizona State University,Online,The Master in Computer Science offered by Cour...
4,Analytics,56726.0,EUR / year,1 year,"Kogod School of Business, American University ...","Washington, D. C., Washington, D.C., United St...","Offered by the Kogod School of Business, the A..."


In [11]:
df.to_excel('{}_{}.xlsx'.format(field, country))